In [1]:
# pip install geopy
import pandas as pd
from geopy.geocoders import ArcGIS
from geopy.extra.rate_limiter import RateLimiter
geolocator = ArcGIS(user_agent='myGeoCoder')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
location = geocode('Rua Clóvis Bevilaqua, 117 - Tijuca, Rio de Janeiro - RJ, 20520-060')

print(type(location))
print(location.address)
print(location.latitude)
print(location.longitude)
print(location.raw)

print()

reverse_geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)
reverse_location = reverse_geocode('-22.929236894906, -43.235731526853')
print(reverse_location)

<class 'geopy.location.Location'>
Rua Clóvis Bevilaqua 117, Tijuca, Rio de Janeiro, 20520-160
-22.929236894906
-43.235731526853
{'address': 'Rua Clóvis Bevilaqua 117, Tijuca, Rio de Janeiro, 20520-160', 'location': {'x': -43.235731526853, 'y': -22.929236894906}, 'score': 99.28, 'attributes': {}, 'extent': {'xmin': -43.236731526853, 'ymin': -22.930236894906, 'xmax': -43.234731526853, 'ymax': -22.928236894906}}

Rua Clóvis Bevilaqua 117, Rio de Janeiro, Rio de Janeiro 20520-160, BRA


In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests
from io import StringIO
import os

def download_kml(url):
    response = requests.get(url)

    if response.status_code == 200:
        return response.content.decode('utf-8')
    else:
        return None

def extract_kml_data(url_or_path: str):
    
    if url_or_path.startswith('https://') or url_or_path.startswith('http://'):
        response = requests.get(url_or_path)
        if response.status_code == 200:
            tree = ET.parse(StringIO(response.content.decode('utf-8')))
        else:
            raise ValueError(f'Erro na tentativa de baixar o arquivo através do link: {url_or_path}')

    elif os.path.exists(url_or_path):
        tree = ET.parse(url_or_path)

    else:
        raise ValueError(f'Entrada inválida! Não foi possível encontrar o arquivo kml: {url_or_path}')
    
    root = tree.getroot()
    namespaces = {'kml': 'http://www.opengis.net/kml/2.2'}
    placemarks = root.findall('.//kml:Placemark', namespaces)

    data_list = []

    for placemark in placemarks:
        data = {}

        for simple_data in placemark.findall('.//kml:SimpleData', namespaces):
            name = simple_data.get('name')
            if name:
                data[name] = simple_data.text

        data_list.append(data)

    return data_list

data_list = extract_kml_data('https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/localidades/cadastro_de_localidades_selecionadas_2010/Google_KML/BR_Localidades_2010_v1.kml')

df = pd.DataFrame(data_list)

print(df.shape)

df.head()

(21886, 21)


,ID,CD_GEOCODIGO,TIPO,CD_GEOCODBA,NM_BAIRRO,CD_GEOCODSD,NM_SUBDISTRITO,CD_GEOCODDS,NM_DISTRITO,CD_GEOCODMU,...,NM_MICRO,NM_MESO,NM_UF,CD_NIVEL,CD_CATEGORIA,NM_CATEGORIA,NM_LOCALIDADE,LONG,LAT,ALT
0,19249,430187505000001,URBANO,None,None,43018750500,None,430187505,BARRA DO QUARAÍ,4301875,...,CAMPANHA OCIDENTAL,SUDOESTE RIO-GRANDENSE,RIO GRANDE DO SUL,1,05,CIDADE,BARRA DO QUARAÍ,-57.5570603248122,-30.2110754079217,42.0408
1,20512,432240005000001,URBANO,432240005001,Centro,43224000500,None,432240005,URUGUAIANA,4322400,...,CAMPANHA OCIDENTAL,SUDOESTE RIO-GRANDENSE,RIO GRANDE DO SUL,1,05,CIDADE,URUGUAIANA,-57.0818249090229,-29.7598231720144,78.23019
2,20041,431530505000001,URBANO,None,None,43153050500,None,431530505,QUARAÍ,4315305,...,CAMPANHA OCIDENTAL,SUDOESTE RIO-GRANDENSE,RIO GRANDE DO SUL,1,05,CIDADE,QUARAÍ,-56.4536470403836,-30.3828679608813,118.674261
3,20167,431710305000001,URBANO,431710305001,Centro,43171030500,None,431710305,SANT'ANA DO LIVRAMENTO,4317103,...,CAMPANHA CENTRAL,SUDOESTE RIO-GRANDENSE,RIO GRANDE DO SUL,1,05,CIDADE,SANT'ANA DO LIVRAMENTO,-55.5348142679597,-30.8893840112129,200.677824
4,19734,431060305000001,URBANO,431060305001,Centro,43106030500,None,431060305,ITAQUI,4310603,...,CAMPANHA OCIDENTAL,SUDOESTE RIO-GRANDENSE,RIO GRANDE DO SUL,1,05,CIDADE,ITAQUI,-56.5571334970304,-29.1286368990609,62.084645
